# AI‑Powered Job Market Insights — Consolidated Analysis

Setup,
Data Load,
EDA,
Modeling (regression, classification, clustering, association rules), Results & Conclusions.


In [ ]:
import os, sys, math, json, itertools, textwrap, warnings, pathlib, statistics
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt

# Modeling
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, classification_report, confusion_matrix

# Clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# Association rules
try:
    from mlxtend.frequent_patterns import apriori, association_rules
except Exception as e:
    print("mlxtend not available. If association rules are needed, install mlxtend.")

warnings.filterwarnings("ignore")

DATA_PATH = "/mnt/data/dataset.csv"


In [ ]:
#Load dataset
df = pd.read_csv("dataset.csv")
print("Rows, Columns:", df.shape)
display(df.head(10))


## Exploratory Data Analysis (from Deliverable 1)

# AI-Powered Job Market Insights: Exploring Automation Risk, Skill Demands, and Future Employment Trends
## Introduction
   The dataset I chose was the "AI-Powered Job Market Insights" from https://www.kaggle.com/datasets/uom190346a/ai-powered-job-market-insights. This dataset provides a synthetic yet highly realistic snapshot of today's evolving job market, with a special emphasis on artificial intelligence (AI) and automation across various industries. This dataset includes 500 records and 10 attributes, such as job title, industry, company size, location, AI adoption level, automation risk, required skills, salary, remote friendliness, and job growth projection. 
   
   I chose this dataset because Ai's impact on the future of work is a widely discussed topic, and I wanted to explore which jobs are most at risk of automation, how AI adoption caries across industries, and what skills are becoming more critical as AI continues to evolve. 
   
   This dataset is well-suited for this project because it meets the requirements of having at least 8-10 attributes and 500+ records, allowing for a comprehensive application of data mining techniques, including data preprocessing, feature engineering, regression modeling, classification, clustering, and association rule mining. This dataset's attributes makes it ideal for uncovering meaningful trends and deriving insights that align with the project's objective of solving real-world problem through data analysis.
    

In [ ]:
#only gives you 5 entries
df.head()

In [ ]:
#gives you a summary of the dataframe
df.info()

We are just gonna see how many of each job titles there are:

In [ ]:
#shows how many of each jobs there are
df.Job_Title.value_counts().sort_values(ascending=False)

Find the total number of mising values in each columns

In [ ]:
df.isnull().sum()

After checking for missing values, we found that the column are complete with no missing data. 

We are going to now remove any duplicates by using the drop_duplicates() method

In [ ]:
df = df.drop_duplicates()

Check for any inconsistent data by using the .unique() method to make sure that values are unique for each column making sure that only what is expected will be printed out

In [ ]:
df['Job_Title'].unique()

In [ ]:
df['Industry'].unique()

In [ ]:
df['Company_Size'].unique()

In [ ]:
df['AI_Adoption_Level'].unique()

In [ ]:
df['Automation_Risk'].unique()

In [ ]:
df['Remote_Friendly'].unique()

In [ ]:
df['Job_Growth_Projection'].unique()

Check to see if there are any inconsistant values in the different columns, for examples if some entries were tech instead of technology. But since the columns are consistant we are good to move on to the next step. 

round to the nearest whole number since more salaries are reported in whole dollars, this makes it easier to read and interpret. Showing the cents doesn't really add meaningful precision.

In [ ]:
df['Salary_USD']=df['Salary_USD'].round()

In [ ]:
df['Salary_USD'].describe()

To check for potential noisy data, we can check for possible outliers. We can use the IQR (interquartile range) rule.

First we will get the IQR with this equation: 

IQR = Q3 - Q1

= 103,971.75 - 78,511.25

= 25,460.50

Then we find the Upper bound: 

Q3 + 1.5 * IQR

= 103,971.75 + (1.5 * 25,460.50)

= 103,971.75 + 38,190.75

= 142,162.50

Then we find the lower bound:

Q1 - 1.5 * IQR

= 78,511.25 - (1.5 * 25,460.50)

= 78,511.25 - 38,190.75

= 40,320.50

The max value which is 155,210 is above the upper bound which could be a possible high-end outlier

The min value which is 31,970 is below the lower bound which could be a possible low-end outlier

These don't necessary mean that these are noisy data, the high-end outlier could be for a executive role and the low-end outlier could be an entry level role.

In [ ]:
#displays a count plot of which industry had the most jobs
plt.figure(figsize=(10, 6))
sns.countplot(y=df['Industry'], order=df['Industry'].value_counts().index)
plt.title('Job Counts by Industry')
plt.xlabel('Count')
plt.ylabel('Industry')
plt.show()

In the graph above it shows that there are alot more job in the Manufacturing industry.

In [ ]:
#displays a count plot of the number of each job title
plt.figure(figsize=(10, 6))
sns.countplot(y=df['Job_Title'], order=df['Job_Title'].value_counts().index)
plt.title('Counts of Job Title')
plt.xlabel('Count')
plt.ylabel('Job Title')
plt.show()

In the graph above it shows that there are alot more jobs as data scientist.

In [ ]:
#counts the number of occurrences of each unique value 
adoption_counts = df['AI_Adoption_Level'].value_counts()

In [ ]:
#creates a new figure
plt.figure(figsize=(6, 6))
#creates a pie chart
plt.pie(adoption_counts, labels=adoption_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of AI Adoption Level')
plt.show()

In [ ]:
#creates a figure for the plot
plt.figure(figsize=(12, 6))
#using seaborns countplot which industry is using AI the most
sns.countplot(data=df, x='AI_Adoption_Level', hue='Industry', order=['Low', 'Medium', 'High'])
plt.title('Distribution of AI Adoption by Industry')
plt.xlabel('AI Adoption Level')
plt.ylabel('Count')
plt.legend(title='Industry', bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
#creates a goruped count plot using seaborn 
sns.countplot(data=df, x='AI_Adoption_Level', hue='Job_Title', order=['Low', 'Medium', 'High'])
plt.title('Distribution of AI Adoption by Job Title')
plt.xlabel('AI Adoption Level')
plt.ylabel('Count')
#adds the key outside of the figure
plt.legend(title='Job Titles',bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
#creates a new dataframe containing only rows where 'Job_Growth_Projection' is 'Growth'
growth_df = df[df['Job_Growth_Projection'] == 'Growth']
plt.figure(figsize=(12, 6))
#creates a count plot showing the number of 'Growth' jobs in each industry
sns.countplot(x='Industry', data=growth_df, order=growth_df['Industry'].value_counts().index)
#adds title
plt.title('Industry Distribution in Jobs with "Growth" Projection')
#add the title for x axis
plt.xlabel('Industry')
#add the title for y axis
plt.ylabel('Count')
#rotates the labels for the x axis so everything fits
plt.xticks(rotation=45)
plt.show()

In [ ]:
growth_df = df[df['Job_Growth_Projection'] == 'Decline']
plt.figure(figsize=(12, 6))
sns.countplot(x='Industry', data=growth_df, order=growth_df['Industry'].value_counts().index)
plt.title('Industry Distribution in Jobs with "Decline" Projection')
plt.xlabel('Industry')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
growth_df = df[df['Job_Growth_Projection'] == 'Growth']
plt.figure(figsize=(12, 6))
sns.countplot(x='Job_Title', data=growth_df, order=growth_df['Job_Title'].value_counts().index)
plt.title('Job Titles with "Growth" Projection')
plt.xlabel('Job Titles')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


In [ ]:
#prints the summary stats for the salary column
print(df['Salary_USD'].describe())

#gives you the first and thrid quartile
Q1 = df['Salary_USD'].quantile(0.25)
Q3 = df['Salary_USD'].quantile(0.75)

#gives the interquartile range
IQR = Q3 - Q1

#gives you the lower and upper bound
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#this identifies the outliers for the salaries 
outliers = df[(df['Salary_USD'] < lower_bound) | (df['Salary_USD'] > upper_bound)]
print(outliers[['Job_Title', 'Industry', 'Salary_USD']])

#plots a boxplot to show salary distribution and outliers
plt.figure(figsize=(8, 5))
sns.boxplot(x=df['Salary_USD'])
plt.title('Salary Boxplot (Detecting Outliers)')
plt.xlabel('Salary (USD)')
plt.show()


In [ ]:
max_salary_row = df[df['Salary_USD'] == df['Salary_USD'].max()]
print(max_salary_row[['Job_Title', 'Industry', 'Location', 'Company_Size', 'Salary_USD']])

There are a couple high-end and low-end outliers that I will need to look into, but for the time being I will assume that because one of the highest paying jobs is in Finance it makes sense.

 From my EDA process I found that most salaries range between $78k and $140K with a few high outliers like Marketing Specialist in Finace at $155k and low outliers like Data Scientist at $32K. Outliers may represent special postions and need careful treatment to avoid skewing models. 
 
 From the job title distribution data scientist is the most common job title, followed by HR Manager, Cybersecurity Analyst, and others. This suggest focus areas for building predicitve models like predicitng salaries or growth for these popular roles.  Job title could be a strong predictive feature for job growth potention.
 
 From the AI adoption distribution, different job titles and industries show varying adoption levels, for example high AI adoption is more common in Technology and Finance and is lower in entertainment and retail. Interaction between AI adoption and industry may affect job risk or growth potential. 
 
 

Since machine learning models operate only on numerical data, feature engineering is essential for converting raw variables into a format that models can interpret effectively. This process plays a critical role in enhancing the model's performance, accuracy, and overall reliability.

In [ ]:
# One-hot encode Industry, Job_Title, AI_Adoption_Level, etc.
df_encoded = pd.get_dummies(df, columns=['Industry', 'Job_Title', 'Company_Size', 
                                         'Location', 'AI_Adoption_Level', 
                                         'Remote_Friendly', 'Job_Growth_Projection'])


In [ ]:
#convert the risk levels to nummerical values where Low = 1, Med = 2, and High = 3
risk_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
df['Automation_Risk_Level'] = df['Automation_Risk'].map(risk_mapping)


In [ ]:
print(df.Automation_Risk_Level)

## Modeling & Evaluation (from Deliverable 2)

## Deliverable 2: Regression Modeling and Performance Evaluation
Anna Bottu

MSCS-634: Advanced Big Data and Data Mining

In [ ]:
#Convert remote-friendliness into a binary feature.
df['Remote_Friendly_Binary'] = df['Remote_Friendly'].map({'Yes': 1, 'No': 0})

In [ ]:
growth_mapping = {'Decline': -1, 'Stable': 0, 'Growth': 1}
df['Job_Growth_Score'] = df['Job_Growth_Projection'].map(growth_mapping)

In [ ]:
df['Skill_Count'] = df['Required_Skills'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

In [ ]:
adoption_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
df['AI_Adoption_Score'] = df['AI_Adoption_Level'].map(adoption_mapping)


From our results we can see that the MAE is 0.71 which means that we are off by 0.71 from the actual job growth score. Since the job growth scores range from -1 to 1, this is a big error. The MSE value just measures larger error more heavily, the value we go was of 0.69, because this is closer to the max target range of 1, this means that there is poor predicition quality. The RMSE value was 0.83 which also tells us that there is a high averge error. Lastly the r^2 value of 0.008, because of this score being so close to 0 this tells us the model barely shows any useful patterns. All these values indicate that these metrics together show us that the AI adoption Score alone is a very poor predictor of job growth in this dataset. 

These r^2 scores are mostly negative or very close to 0, which means that the model's performace is unstable and does not generalize well. Due to our mean being negative, that suggests that the model's predictions are less accurate.

In [ ]:
from sklearn.model_selection import train_test_split

# Features and target
X = df[['AI_Adoption_Score']]   # predictor
y = df['Job_Growth_Score']      # target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Predictions
y_pred_lin = lin_reg.predict(X_test)

# Create jittered version of AI_Adoption_Score
jittered_x = X_test['AI_Adoption_Score'] + np.random.normal(0, 0.1, size=len(X_test))

# Plot
plt.figure(figsize=(8, 5))
sns.scatterplot(x=jittered_x, y=y_test, color='black', label='Actual (jittered)')
sns.lineplot(x=X_test['AI_Adoption_Score'], y=y_pred_lin, color='red', label='Predicted')
plt.title('Linear Regression: AI Adoption vs Job Growth')
plt.xlabel('AI Adoption Score')
plt.ylabel('Job Growth Score')
plt.legend()
plt.grid(True)
plt.xlim(0.5, 3.5)   # expand x-axis
plt.ylim(-1.2, 1.2)  # expand y-axis
plt.show()


As you can see thse scores are similar to the linear regession model results.

In [ ]:
# Imports you need for these cells
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 1) Features & target (adjust column names if yours differ)
X = df[['AI_Adoption_Score']]
y = df['Job_Growth_Score']

# 2) Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3) Fit models
lin_reg = LinearRegression().fit(X_train, y_train)

ridge = Pipeline([
    ('scaler', StandardScaler()),   # scale single feature before Ridge
    ('reg', Ridge(alpha=1.0, random_state=42))
]).fit(X_train, y_train)

# 4) Predictions (these are what your plot uses)
y_pred_lin   = lin_reg.predict(X_test)
y_pred_ridge = ridge.predict(X_test)

# 5) Jitter for the scatter (re-use in both plots)
jittered_x = X_test['AI_Adoption_Score'] + np.random.normal(0, 0.1, size=len(X_test))
plt.figure(figsize=(8, 5))
sns.scatterplot(x=jittered_x, y=y_test, color='black', label='Actual (jittered)')
sns.lineplot(x=X_test['AI_Adoption_Score'], y=y_pred_ridge, color='red', label='Predicted (Ridge)')
plt.title('Ridge Regression: AI Adoption vs Job Growth')
plt.xlabel('AI Adoption Score')
plt.ylabel('Job Growth Score')
plt.grid(True)
plt.legend()
plt.xlim(0.5, 3.5)
plt.ylim(-1.2, 1.2)
plt.show()

I explored whether AI adoption predicts job growth potential using two regression models, Linear Regression and Ridge Regression. Although my primary interest was in identifying correlation, building regression models helped confirm the relationship was weak.

Both regression models, Linear and Ridge, were used to evaluate the potential relationship between AI adoption and job growth. The r^2 scores for both models were near zero or negative, and cross-validation confirmed very poor generalization. This demonstrates that AI adoption level alone is not a strong predictor of job growth in this dataset. Therefore, while the models fulfill the requirement of regression analysis, they collectively reinforce our key insight: there is no significant correlation between AI adoption and job growth based on the available data.

Two regression models, the Linear Regression and Ridge Regression models were developed to examine whether AI adoption levels could predict job growth potential. The Linear Regression model yielded a mean absolute error (MAE) of 0.71, a mean squared error (MSE) of 0.69, a root mean squared error (RMSE) of 0.83, and a very low r^2 value of 0.008. Ridge Regression model yielded a MAE of 0.71, a MSE of 0.69, a RMSE of 0.83 and also a very low r^2 value of 0.008, which incorporates regularization to mitigate overfitting, produced similar results. Both models also demonstrated low or negative r^2 scores during cross-validation, indicating poor generalization to unseen data.

These evaluation results suggest that AI Adoption Score alone does not serve as a meaningful predictor of job growth in this dataset. Overall, the analysis indicates that more complex modeling using additional features may be necessary to effectively predict job growth, as AI adoption on its own does not sufficiently explain the variation in growth outcomes.

## Clustering & Association Rules (from Deliverable 3)

The decision tress classifier performed poorly in predicting the job growth potential projections, achieving only 35% accuracy, which is close to random guessing across the three classes (low, medium, and high). The confusion matrix shows frequent misclassification, especially between classes 0 and 2, suggesting that the features used do not provide strong separation for this target and that more advanced models or better feature engineering may be needed.

The k-NN classifier achieved an accuracy of 37%, which is only slightly better than random guessing across the three job growth projection classes. The confusion matrix shows that although the model correctly shows some instances particularly in Class 2, it frequently misclassified jobs across all categories, with overlap between Classes 0, 1, and 2. Overall, this indicates that k-NN struggles to separate the job growth projection groups, suggesting that the available features may not provide enough distinction for reliable predictions.

The tuned decision tree classifier achieved a test accuracy of 35%, which is similar to the untuned version and only a little better than random guessing across the three job growth projection classes. Even after hyperparameter optimization, the confusion matrix shows frequent misclassifications, particularly between Classes 0 and 2, showing that the features used do not provide enough separation for accurate predictions. This indicates that decision trees, even when tuned, may not be the best model for this dataset, and more advanced methods or feature engineering may be needed to improve performance.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

def evaluate_model(model, X_test, y_test, model_name="Model"):
    """
    Evaluate a classification model on test data.
    Prints Accuracy, F1 score, and Classification Report.
    """
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, average="weighted")  # weighted for multi-class

    print(f"--- {model_name} ---")
    print(f"Accuracy: {acc:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("-"*40)

    return acc, f1


These results show that the tuned decision tree performed poorly, with an accuracy of about 35% and an F1 score of 0.35, which is only a little better than random guessing across the three job growth projection classes. The k-NN model performed a little better, reaching 41% accuracy and an F1 score of 0.40, but it still struggled to correctly show the difference between the classes, as seen in the confusion matrix where many predictions overlap. Overall, both models show that the current features do not give us a good prediction for job growth projection, and more advanced models or additional features may be needed to improve performance.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Pick numeric features that actually exist
X_features = df[['AI_Adoption_Level', 'Automation_Risk', 'Salary_USD', 'Job_Growth_Projection']]

# If any of these have missing values, fill or drop (simple fill here)
X_features = X_features.fillna(X_features.median(numeric_only=True))

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Elbow method
inertia = []
K_range = range(1, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(K_range, inertia, marker="o")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia (Within-Cluster SSE)")
plt.title("Elbow Method for Optimal k")
plt.show()


The elbow method plot shows how the within-cluster variance decreases as the number of clusters increases. The curve starts to flatten around k = 3, which tells us that three clusters may be the best choice, as adding more clusters beyond that point will give us only small improvements. Grouping the data into three clusters gives a good balance between efficiency and accuracy, capturing the main structure of the dataset without overcomplicating it.

In [ ]:

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

df["Cluster"] = clusters

print(df.groupby("Cluster").mean())  # See feature patterns per cluster


This K-Means model divided the dataset into three clusters, each with its own characteristics. Cluster 0 is made up of jobs with moderate salaries, slightly higher automation risk, and limited remote opportunities. Cluster 1 stands out with higher salaries, more specialized skill requirements, and the tendency to be remote-friendly, telling us that these are higher-skill roles with better flexibility. Cluster 2 also shows relatively strong salaries and advanced skill needs, but unlike Cluster 1, these jobs are less likely to be remote-friendly. The results tells us that while Clusters 1 and 2 have higher-paying, skill-intensive roles, Cluster 1 has the added advantage of remote work, making it attractive in today’s job market.

This visualization shows how the K-Means algorithm grouped the data into three distinct clusters after reducing the dimensions with PCA. Each color represents a cluster, and we can see that the points are well separated, which means the algorithm was able to detect patterns in the data. It confirms that the dataset naturally splits into three groups with different feature characteristics.

In [ ]:
# Look at average feature values per cluster
cluster_summary = df.groupby("Cluster").mean()
print(cluster_summary)


In [ ]:
import seaborn as sns
plt.figure(figsize=(8,5))
sns.boxplot(x="Cluster", y="Salary_USD", data=df)
plt.title("Salary Distribution by Cluster")
plt.show()

sns.countplot(x="Cluster", hue="Automation_Risk", data=df)
plt.title("Automation Risk by Cluster")
plt.show()


The clustering results shows three main groups of jobs that differ in salary levels and automation risk. Cluster 0 includes mid-level positions with salaries around 80,000–90,000, but with a lot of variation depending on the industry. These jobs fall across both stable roles and ones more vulnerable to automation. Cluster 1 stands out with higher salaries, often over 100,000, and lower automation risk, likely representing specialized, high-skill positions in fields like technology or management where pay is strong and job security is better. Cluster 2 is more mixed, with some well-paying roles but also many lower-salary jobs, and it carries a higher risk of automation. This cluster likely reflects roles in areas like sales, marketing, or operations, which may pay competitively now but could face challenges as automation grows. Overall, Cluster 1 appears to offer the best balance of pay and stability, while Cluster 2 highlights jobs where reskilling may be important in the future.

In [ ]:
!pip install mlxtend


In [ ]:
pip install --upgrade pip setuptools packaging


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# pick the categorial columns for the association rule mining
cat_cols = [
    "Industry","Company_Size","Location",
    "AI_Adoption_Level","Automation_Risk",
    "Required_Skills","Remote_Friendly","Job_Growth_Projection"
]
#transforms the data into true/false values
transactions = pd.get_dummies(df[cat_cols], drop_first=False).astype(bool)

#makes sure that the dataset only has true and false
assert set(transactions.stack().unique()) <= {True, False}



In [ ]:
#frequent itemsets
frequent_itemsets = apriori(transactions, min_support=0.05, use_colnames=True)

#association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules = rules.sort_values("lift", ascending=False)

rules[['antecedents','consequents','support','confidence','lift']].head(10)


The patterns uncovered through association rule mining show us several important trends that connect job characteristics with industry outcomes. One of the strongest findings is that remote-friendly jobs requiring specialized skills are more likely to be found in industries that are projected to grow. This suggests that for individuals, developing these in-demand skills can open up opportunities for stable, flexible careers, while for employers, promoting remote options can help attract talent in competitive fields. Another key insight is the strong link between AI adoption and industry expansion. Companies that invest in AI, especially those offering remote work and requiring advanced skills, are concentrated in industries with positive job growth projections. This has real implications where workers in high-automation-risk jobs can look toward reskilling opportunities in these areas, while companies can use this knowledge to plan retraining programs rather than downsizing. Larger companies with higher AI adoption also tend to demand more advanced technical and digital skills, which signals to job seekers the value of certifications in areas like cloud computing, cybersecurity, and AI. Taken together, these patterns emphasize that remote work, AI adoption, and advanced skill sets are strong indicators of where the most resilient and rewarding jobs are likely to be found, while also underscoring the need for adaptation and reskilling in roles more vulnerable to automation.